In [14]:
import rouge_score

In [77]:
import pandas as pd 
import numpy as np
import nltk
from nltk.corpus import stopwords
from tqdm._tqdm_notebook import tqdm_notebook
import warnings
from nltk.tokenize import sent_tokenize

nltk.download('punkt')
warnings.filterwarnings('ignore')
tqdm_notebook.pandas()
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dfm4ik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
df = pd.read_csv('cnn_dailymail/train.csv')

In [34]:
rouge_metrics = rouge_score.rouge_scorer.RougeScorer(['rouge1', 'rougeL','rouge2'], use_stemmer=True)

In [53]:
df.head()

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [54]:
df['article'][0]

"By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A . State Immunization Program Manager Molly Howell says the risk is low, but officials feel it's important to alert people to the possible exposure. The diocese announced on Monday that Bishop John Folda is taking time off after being diagnosed with hepatitis A. The diocese says he contracted the infection through contaminated food while attending a conference for newly ordained 

In [56]:
print(df['highlights'][0])

Bishop John Folda, of North Dakota, is taking time off after being diagnosed .
He contracted the infection through contaminated food in Italy .
Church members in Fargo, Grand Forks and Jamestown could have been exposed .


In [57]:
df.shape

(287113, 3)

In [67]:
df_need = df[:1000]

In [ ]:
PERCENT_RATIO = 0.2
STOPWORDS = stopwords.words('english')

#### text rank

In [ ]:
from summa.summarizer import summarize as textRankSumm

In [78]:
df_need['text_rank'] = df_need['article'].progress_apply(lambda x:summarize(x,ratio = PERCENT_RATIO,additional_stopwords = STOPWORDS))

  0%|          | 0/1000 [00:00<?, ?it/s]

#### LexRank

In [124]:
from lexrank import LexRank

lxr = LexRank(df_need['article'],stopwords=STOPWORDS)
df_need['article_sentences'] = df_need['article'].apply(lambda x:sent_tokenize(x))


In [127]:
df_need['lex_rank'] = df_need['article_sentences']\
.progress_apply(lambda x: lxr\
                .get_summary(x,
                             summary_size=round(PERCENT_RATIO*len(x))
                            )
               )

  0%|          | 0/1000 [00:00<?, ?it/s]

In [128]:
def connect(x:list[str])->str:
    return ''.join(x)

df_need['lex_rank'] = df_need['lex_rank'].apply(conect)

#### Evaluate



In [134]:
print(df_need['text_rank'][0],'\n\n',df_need['highlights'][0])

The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October.
Bishop John Folda (pictured) of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A . 

 Bishop John Folda, of North Dakota, is taking time off after being diagnosed .
He contracted the infection through contaminated food in Italy .
Church members in Fargo, Grand Forks and Jamestown could have been exposed .


In [141]:
abba = rouge_metrics.score(df_need['highlights'][0],df_need['text_rank'][0])

In [144]:
abba['rougeL'].fmeasure

0.28865979381443296

In [153]:
df_need.columns

Index(['id', 'article', 'highlights', 'text_rank', 'article_sentences',
       'lex_rank'],
      dtype='object')

In [164]:

df_need['text_rank_scores'] = df_need[['highlights','text_rank']]\
.apply(lambda x: rouge_metrics.score(x['highlights'],x['text_rank'])['rougeL'].fmeasure,axis = 1)

df_need['lex_rank_scores'] = df_need[['highlights','lex_rank']]\
.apply(lambda x: rouge_metrics.score(x['highlights'],x['lex_rank'])['rougeL'].fmeasure,axis = 1)

In [162]:
df_need['text_rank_scores']

0      0.288660
1      0.126582
2      0.224924
3      0.119403
4      0.099099
         ...   
995    0.140000
996    0.176166
997    0.194175
998    0.283333
999    0.183673
Name: text_rank_scores, Length: 1000, dtype: float64

In [165]:
df_need['lex_rank_scores'] 

0      0.241379
1      0.189189
2      0.139394
3      0.216749
4      0.118644
         ...   
995    0.126984
996    0.251429
997    0.202899
998    0.216216
999    0.234234
Name: lex_rank_scores, Length: 1000, dtype: float64

In [170]:
import numpy as np
from scipy import stats

def calculate_confidence_interval(data, confidence=0.95):
    mean = np.mean(data)
    n = len(data)
    stderr = stats.sem(data)
    interval = stderr * stats.t.ppf((1 + confidence) / 2., n - 1)
    return mean - interval, mean ,mean + interval

In [171]:
calculate_confidence_interval(df_need['lex_rank_scores'])

(0.15897700141626406, 0.16372254932094313, 0.1684680972256222)

In [172]:
calculate_confidence_interval(df_need['text_rank_scores'])

(0.1724774936049619, 0.17748832458745759, 0.18249915556995328)